In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import random

In [2]:
class gridworld_env:
    def __init__(self, x, y):
        
        # The board of the grid world
        self.world = np.arange(0,25).reshape((5,5))
        
        # Current position of the agent on the board
        self.current_position = (x,y)
        
        # Gamma coefficient
        self.gamma = 0.9
        
        # Possible moves
        self.WEST = 0
        self.NORTH = 1
        self.EAST = 2
        self.SOUTH = 3
        self.action_spaces = [self.WEST, self.NORTH, self.EAST, self.SOUTH]
        
        # Special states
        self.aprime = (0,1)
        self.bprime = (0,3)
    '''
    Agent take an action to move to new state, the environment returns 
    the reward for the action
    '''
    def take_action(self, action):
        
        # Compute the reward and new position of the agent for the action
        reward = 0
        new_position = self.current_position
        if action == self.NORTH:
            new_position = (max(self.current_position[0]-1,0), self.current_position[1])
            if new_position == self.current_position:
                reward = -1
        elif action == self.WEST:
            new_position = (self.current_position[0], max(self.current_position[1]-1,0))
            if new_position == self.current_position:
                reward = -1
        elif action == self.EAST:
            new_position = (self.current_position[0], (self.current_position[1]+1)%5)
            if new_position[1] == 0:
                reward = -1
                new_position = self.current_position
        elif action == self.SOUTH:
            new_position = ((self.current_position[0]+1)%5, self.current_position[1])
            if new_position[0] == 0:
                reward = -1
                new_position = self.current_position
        
        # Special reward if the agent is in A prime or B prime, then
        # we can ignore the computation of the reward above
        if self.current_position == self.aprime:
            reward = 10
            new_position = (4,1)
        elif self.current_position == self.bprime:
            reward = 5
            new_position = (2,3)
            
        # Create a new state of the gridworld
        new_env = gridworld_env(new_position[0], new_position[1])
        
        # Return the reward of the action and the new state of the gridworld
        return reward, new_env

#### Implementation of the Monte Carlo Algorithm Taken from The Book

In [17]:
def naive_monte_carlo_algorithm(episodes, ep_length):
    
    # The state value of each state
    ex_rewards = np.zeros((5,5))
    reward_of_state = [[np.array([]) for j in range(0,5)] for i in range(0, 5)]

    # Number of steps in each episode
    time = ep_length

    # Loop for each episode
    for episode in range(0, episodes):
    
        # Create a grid world environment
        env = gridworld_env(random.randint(0,4), random.randint(0,4))
    
        # The sequence of states and reward with respect to time t
        states_sequence = [0] * time
        reward_sequence = [0] * time
    
        # Generate a sequence of states and rewards
        for t in range(0, time):
            # Take a random action
            action = random.randint(0,4)
            reward, env = env.take_action(action)
        
            # Store the reward sequence and state sequence
            states_sequence[t] = env.current_position
            reward_sequence[t] = reward
    
        # Cumulative reward G
        total_reward = 0
    
        # Loop for each step of episode, t=T−1,T−2,...,0
        for t in range(time-1, -1, -1):
        
            # Get the cumulative reward
            total_reward += reward_sequence[t]
        
            # Check if S_t has appeared in the sequence S_0, S_1, ..., S_t-1 or not
            tmp = states_sequence[:t]
        
            # If the new state hasn't been explored yet:
            if states_sequence[t] not in tmp:
                returns_st = reward_of_state[states_sequence[t][0]][states_sequence[t][1]]
                reward_of_state[states_sequence[t][0]][states_sequence[t][1]] = np.append(returns_st, total_reward)
            
                ex_rewards[states_sequence[t][0], states_sequence[t][1]] = np.average(returns_st)
                
    return ex_rewards

In [16]:
naive_monte_carlo_algorithm(100000, 50)

/Users/user/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:1110: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/user/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


array([[ 3.80924964,  8.00441173,  4.9778377 ,  5.45754813,  2.06795525],
       [ 2.14868135,  3.46962311,  2.93941561,  2.48592095,  1.0788098 ],
       [ 0.31546066,  1.02142393,  0.98263021,  1.984307  , -0.30994815],
       [-1.27444658, -0.70573769, -0.60383337, -0.95304348, -1.62473246],
       [-2.56030312,  2.18911084, -1.88304962, -2.14988013, -2.74986633]])